In [1]:
import pandas as pd
import numpy as np
# 数据加载
data = pd.read_csv('./family_data.csv')
data

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
0,0,52,38,12,82,33,75,64,76,10,28,4
1,1,26,4,82,5,11,47,38,6,66,61,4
2,2,100,54,25,12,27,82,10,89,80,33,3
3,3,2,95,1,96,32,6,40,31,9,59,2
4,4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,16,1,66,33,18,70,56,46,86,60,4
4996,4996,88,66,20,17,26,54,81,91,59,48,2
4997,4997,32,66,54,17,27,21,74,81,3,7,6
4998,4998,67,92,4,17,53,77,1,12,26,70,5


## 数据预处理
* 1）计算Perference Cost矩阵 pcost_mat
* 2）计算Accounting Cost矩阵 acost_mat
* 3）计算每个家庭的人数 FAMILY_SIZE
* 4）每个家庭的倾向选择（choice_） DESIRED

In [2]:
# 设置常数
N_DAYS = 100 # 一共安排的天数
N_FAMILY = 5000 # 一共接待的家庭数量
MIN_OCCUPANCY = 125 # 最少接待人数
MAX_OCCUPANCY = 300 # 最大接待人数
FAMILY_SIZE = data['n_people'].values # 每个家庭的成员数量
DESIRED = data.iloc[:,1:-1].values-1 # 每个家庭的选择倾向，下标从0开始，需要减一

In [3]:
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4], dtype=int64)

In [4]:
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]], dtype=int64)

In [5]:
def get_penalty(n,choice):
    '''
        n代表家庭成员的数量
        choice代表santa满足该家庭的第几个choice（0-9）
    '''
    penalty = 0
    if choice == 0:
        pass
    elif choice == 1:
        penalty = 50
    elif choice == 2:
        penalty = 50 + 9*n
    elif choice == 3:
        penalty = 100 + 9*n
    elif choice == 4:
        penalty = 200 + 9*n
    elif choice == 5:
        penalty = 200 + 18*n
    elif choice == 6:
        penalty = 300 + 18*n
    elif choice == 7:
        penalty = 300 + 36*n
    elif choice == 8:
        penalty = 400 + 36*n    
    elif choice == 9:
        penalty = 500 + (36+199)*n
    else:
        penalty = 500 + (36+398)*n 
    return penalty

In [6]:
# 计算pcost_mat,代表某个家庭，在某天接待时的penalty
pcost_mat = np.full(shape=(N_FAMILY,N_DAYS),fill_value=np.NaN)
pcost_mat.shape

(5000, 100)

In [7]:
for f in range(N_FAMILY):
    f_num = data.loc[f,'n_people']
    # 先按照otherwise进行初始化:100天内，只有10个choice，大部分都是otherwise
    pcost_mat[f,:] = get_penalty(f_num,10)
    for choice in range(10):
        # 先找到家庭f在choice时的选择 => temp
        temp = data.loc[f,'choice_'+str(choice)]
        penalty = get_penalty(f_num,choice)
        # 更新家庭f在temp-1天的penalty
        pcost_mat[f,temp-1] = penalty
        
print(pcost_mat)

[[2236. 2236. 2236. ... 2236. 2236. 2236.]
 [2236. 2236. 2236. ... 2236. 2236. 2236.]
 [1802. 1802. 1802. ... 1802. 1802.    0.]
 ...
 [3104. 3104.  616. ... 3104. 3104. 3104.]
 [ 390. 2670. 2670. ... 2670. 2670. 2670.]
 [2236. 2236. 2236. ... 2236. 2236. 2236.]]


In [8]:
# 计算Accounting Penalty矩阵。参数：当天认识，前一天人数。假设最大安排300人
acost_mat = np.zeros(shape=(MAX_OCCUPANCY,MAX_OCCUPANCY))
acost_mat.shape

(300, 300)

In [9]:
# 当天安排的人数-1
for i in range(MAX_OCCUPANCY):
    # 前一天安排的人数-1
    for j in range(MAX_OCCUPANCY):
        diff = abs(i-j)
        acost_mat[i,j] = (i+1-MIN_OCCUPANCY)/400*((i+1)**(0.5+diff/50))
acost_mat        

array([[-3.10000000e-01, -3.10000000e-01, -3.10000000e-01, ...,
        -3.10000000e-01, -3.10000000e-01, -3.10000000e-01],
       [-4.40941239e-01, -4.34870670e-01, -4.40941239e-01, ...,
        -2.63304143e+01, -2.66979732e+01, -2.70706630e+01],
       [-5.52007930e-01, -5.40011355e-01, -5.28275496e-01, ...,
        -3.45045104e+02, -3.52710424e+02, -3.60546032e+02],
       ...,
       [ 3.71482922e+15,  3.31477861e+15,  2.95780952e+15, ...,
         7.46610759e+00,  8.36716954e+00,  9.37697794e+00],
       [ 4.27883100e+15,  3.81778713e+15,  3.40642072e+15, ...,
         8.43020770e+00,  7.52185316e+00,  8.43020770e+00],
       [ 4.92860244e+15,  4.39725208e+15,  3.92318635e+15, ...,
         9.51970597e+00,  8.49339085e+00,  7.57772228e+00]])

In [10]:
# eg：当天接待126人，前一天接待127人的penalty
print(acost_mat[126-1,127-1])
# 当天接待126人，前一天接待300人的penalty
print(acost_mat[126-1,300-1])

0.030912397699621216
572024.6950297297


## 使用LP和IP求解 规划方案
* 1）先使用LP 对绝大部分家庭进行规划
* 2）再使用IP 对剩余家庭进行规划
* 3）汇总两边的结果 => 最终规划方案

# 使用LP快速求解

In [11]:
from ortools.linear_solver import pywraplp
# 创建LP求解器
# solver = pywraplp.Solver.CreateSolver('SCIP')
solver= pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING) 
# solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAM MING)
# solver = pywraplp.Solver.CreateSolver('CBC') 

# 每一天都有哪些家庭
candidates = [[] for x in range(N_DAYS)]

# 定义决策变量x
x = {}
# i是family_id
for i in range(N_FAMILY):
    # j是对应家庭的10个选项（j的范围是1-100）
    for j in DESIRED[i,:]:
        # 第j+1天都有哪些家庭
        candidates[j].append(i)
        # 定义决策变量x[i,j],i代表famil_id，j代表第j+1天参观
        x[i,j] = solver.BoolVar(f'x[{i},{j}]') # 每个家庭只有10个choice
        
# 最大维度5000*100，但是只有5000*10是有数值的，放到字典x中
x

{(0, 51): x[0,51],
 (0, 37): x[0,37],
 (0, 11): x[0,11],
 (0, 81): x[0,81],
 (0, 32): x[0,32],
 (0, 74): x[0,74],
 (0, 63): x[0,63],
 (0, 75): x[0,75],
 (0, 9): x[0,9],
 (0, 27): x[0,27],
 (1, 25): x[1,25],
 (1, 3): x[1,3],
 (1, 81): x[1,81],
 (1, 4): x[1,4],
 (1, 10): x[1,10],
 (1, 46): x[1,46],
 (1, 37): x[1,37],
 (1, 5): x[1,5],
 (1, 65): x[1,65],
 (1, 60): x[1,60],
 (2, 99): x[2,99],
 (2, 53): x[2,53],
 (2, 24): x[2,24],
 (2, 11): x[2,11],
 (2, 26): x[2,26],
 (2, 81): x[2,81],
 (2, 9): x[2,9],
 (2, 88): x[2,88],
 (2, 79): x[2,79],
 (2, 32): x[2,32],
 (3, 1): x[3,1],
 (3, 94): x[3,94],
 (3, 0): x[3,0],
 (3, 95): x[3,95],
 (3, 31): x[3,31],
 (3, 5): x[3,5],
 (3, 39): x[3,39],
 (3, 30): x[3,30],
 (3, 8): x[3,8],
 (3, 58): x[3,58],
 (4, 52): x[4,52],
 (4, 0): x[4,0],
 (4, 46): x[4,46],
 (4, 92): x[4,92],
 (4, 25): x[4,25],
 (4, 2): x[4,2],
 (4, 45): x[4,45],
 (4, 15): x[4,15],
 (4, 41): x[4,41],
 (4, 38): x[4,38],
 (5, 31): x[5,31],
 (5, 58): x[5,58],
 (5, 11): x[5,11],
 (5, 2): x[5,2]

In [12]:
# 统计每天接待的人数,i代表family_id
daily_occupancy = [solver.Sum(
    [x[i,j]*FAMILY_SIZE[i] for i in candidates[j]])
    for j in range(N_DAYS)]

# 每天运载能力的约束
for j in range(N_DAYS):
    solver.Add(daily_occupancy[j]>=MIN_OCCUPANCY)
    solver.Add(daily_occupancy[j]<=MAX_OCCUPANCY)
    
daily_occupancy

In [13]:
# 对于accounting_cost进行简化约束，即前后两天的人数差不超过20(手动测出来的值)
for j in range(N_DAYS-1):
    solver.Add(daily_occupancy[j]-daily_occupancy[j+1] <= 20)
    solver.Add(daily_occupancy[j+1]-daily_occupancy[j] <= 20)

In [14]:
# 统计每个家庭在10个choice中出现的情况
family_presence = [solver.Sum([x[i,j] for j in DESIRED[i,:]]) 
                            for i in range(N_FAMILY)]

# 约束：每个家庭在10个choice有且只出现一次
for i in range(N_FAMILY):
    # i是family_id
    solver.Add(family_presence[i]==1)
    
family_presence

 ...]

In [15]:
# 定义目标函数pcost
preference_cost = solver.Sum([pcost_mat[i,j]*x[i,j] for i in range(N_FAMILY) for j in DESIRED[i,:]])

preference_cost

In [16]:
%time
# 先满足preference_cost最小
solver.Minimize(preference_cost)
status = solver.Solve()
status

Wall time: 0 ns


0

In [17]:
# Solve的结果,x[i,j]决策变量代表第i个family在第j天是否参观
result = [(i,j,x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:]
             if x[i,j].solution_value() > 0]
result

[(0, 51, 1.0),
 (1, 25, 1.0),
 (2, 99, 0.9999999999999999),
 (3, 1, 1.0),
 (4, 52, 1.0),
 (5, 31, 1.0),
 (6, 87, 1.0),
 (7, 24, 1.0),
 (8, 17, 1.0),
 (9, 49, 1.0),
 (10, 91, 1.0),
 (11, 18, 1.0),
 (12, 97, 1.0),
 (13, 53, 1.0),
 (14, 44, 1.0),
 (15, 21, 1.0),
 (16, 49, 0.9999999999999999),
 (17, 46, 1.0),
 (18, 74, 0.9999999999999999),
 (19, 2, 1.0),
 (20, 2, 1.0),
 (21, 55, 1.0),
 (22, 60, 1.0),
 (23, 18, 0.9999999999999999),
 (24, 74, 1.0),
 (25, 15, 0.9999999999999999),
 (26, 57, 1.0),
 (27, 37, 1.0),
 (28, 26, 1.0),
 (29, 7, 1.0),
 (30, 67, 0.9999999999999999),
 (31, 73, 0.9999999999999999),
 (32, 23, 1.0),
 (33, 31, 1.0),
 (34, 45, 0.9999999999999999),
 (35, 30, 1.0),
 (36, 46, 1.0),
 (37, 14, 1.0),
 (38, 12, 1.0),
 (39, 8, 1.0),
 (40, 24, 1.0),
 (41, 91, 1.0),
 (42, 80, 1.0),
 (43, 10, 1.0),
 (44, 48, 1.0),
 (45, 4, 1.0),
 (46, 14, 1.0),
 (47, 44, 1.0),
 (48, 31, 1.0),
 (49, 32, 1.0),
 (50, 49, 1.0),
 (51, 27, 1.0),
 (52, 32, 1.0),
 (53, 36, 1.0),
 (54, 68, 0.9999999999999999),
 

In [18]:
# 得到参观日期的安排
df = pd.DataFrame(result,columns=['family_id','day','result'])
df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5072,4995,15,1.0
5073,4996,87,1.0
5074,4997,31,1.0
5075,4998,91,1.0


In [19]:
df['family_id'].value_counts()

4045    2
2778    2
2289    2
244     2
248     2
       ..
1682    1
1681    1
1680    1
1679    1
4999    1
Name: family_id, Length: 5000, dtype: int64

In [20]:
# 设置阈值
THRESHOLD=0.999
# 已经安排的家庭
assigned_df = df[df['result']>THRESHOLD]
# 尚未安排的家庭
unassigned_df = df[(df['result']<=THRESHOLD) & (df['result']>=1-THRESHOLD)]
unassigned_df

,family_id,day,result
97,97,95,0.500000
98,97,47,0.500000
235,233,86,0.333333
236,233,91,0.666667
247,244,38,0.750000
...,...,...,...
4993,4918,42,0.285714
5011,4936,73,0.500000
5012,4936,61,0.500000
5055,4979,13,0.666667


In [21]:
df[df['family_id']==1653]

,family_id,day,result
1681,1653,5,0.5
1682,1653,96,0.5


In [22]:
# 4945个家庭可以直接安排上，剩余55个需要调整
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5072,4995,15,1.0
5073,4996,87,1.0
5074,4997,31,1.0
5075,4998,91,1.0


In [23]:
# 添加family_size字段
assigned_df['family_size'] = FAMILY_SIZE[assigned_df['family_id']]
assigned_df

C:\Users\wtxu3\AppData\Local\Temp/ipykernel_19800/1404069838.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assigned_df['family_size'] = FAMILY_SIZE[assigned_df['family_id']]


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5072,4995,15,1.0,4
5073,4996,87,1.0,2
5074,4997,31,1.0,6
5075,4998,91,1.0,5


In [24]:
# 按照day进行聚合
occupancy = assigned_df.groupby('day')['family_size'].sum().values
occupancy

array([290, 277, 298, 297, 278, 254, 238, 260, 278, 294, 299, 297, 275,
       254, 257, 280, 298, 286, 271, 254, 232, 237, 264, 285, 296, 297,
       279, 253, 229, 255, 276, 272, 252, 232, 209, 193, 215, 229, 250,
       231, 214, 192, 208, 230, 252, 269, 246, 230, 212, 206, 228, 248,
       246, 225, 207, 188, 169, 191, 204, 189, 169, 146, 124, 120, 137,
       165, 174, 156, 136, 124, 125, 136, 158, 169, 151, 138, 125, 125,
       142, 162, 183, 158, 140, 120, 132, 148, 171, 189, 174, 152, 130,
       123, 130, 153, 174, 152, 130, 123, 122, 123], dtype=int64)

In [25]:
min_occupancy = np.array([max(0,MIN_OCCUPANCY-x) for x in occupancy])
min_occupancy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 3, 2], dtype=int64)

In [26]:
max_occupancy = np.array([(MAX_OCCUPANCY-x) for x in occupancy])
max_occupancy

array([ 10,  23,   2,   3,  22,  46,  62,  40,  22,   6,   1,   3,  25,
        46,  43,  20,   2,  14,  29,  46,  68,  63,  36,  15,   4,   3,
        21,  47,  71,  45,  24,  28,  48,  68,  91, 107,  85,  71,  50,
        69,  86, 108,  92,  70,  48,  31,  54,  70,  88,  94,  72,  52,
        54,  75,  93, 112, 131, 109,  96, 111, 131, 154, 176, 180, 163,
       135, 126, 144, 164, 176, 175, 164, 142, 131, 149, 162, 175, 175,
       158, 138, 117, 142, 160, 180, 168, 152, 129, 111, 126, 148, 170,
       177, 170, 147, 126, 148, 170, 177, 178, 177], dtype=int64)

# IP规划

In [27]:
def solveIP(families,min_occupancy,max_occupancy):
    '''
        families:list 有哪些家庭需要被安排
        min_occupancy:list
        max_occupancy:list
    '''
    # 创建IP求解器
    solver = pywraplp.Solver.CreateSolver('CBC') 
    # 需要安排的家庭
    n_family = len(families)
    # 每一天都有哪些家庭
    candidates = [[] for x in range(N_DAYS)]

    # 定义决策变量x
    x = {}
    # i是family_id
    for i in families:
        # j是对应家庭的10个选项（j的范围是1-100）
        for j in DESIRED[i,:]:
            # 第j+1天都有哪些家庭
            candidates[j].append(i)
            # 定义决策变量x[i,j],i代表famil_id，j代表第j+1天参观
            x[i,j] = solver.BoolVar(f'x[{i},{j}]') # 每个家庭只有10个choice

    # 统计每天接待的人数,i代表family_id
    daily_occupancy = [solver.Sum(
        [x[i,j]*FAMILY_SIZE[i] for i in candidates[j]])
        for j in range(N_DAYS)]

    # 每天运载能力的约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=min_occupancy[j])
        solver.Add(daily_occupancy[j]<=max_occupancy[j])

    # 对于accounting_cost进行简化约束，即前后两天的人数差不超过20(手动测出来的值)
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j]-daily_occupancy[j+1] <= 20)
        solver.Add(daily_occupancy[j+1]-daily_occupancy[j] <= 20) 
        
        
    # 统计每个家庭在10个choice中出现的情况
    family_presence = [solver.Sum([x[i,j] for j in DESIRED[i,:]]) 
                                for i in families]

    # 约束：每个家庭在10个choice有且只出现一次
    for i in range(n_family):
        # i是family_id
        solver.Add(family_presence[i]==1)
       
        
    # 定义目标函数pcost
    preference_cost = solver.Sum([pcost_mat[i,j]*x[i,j] for i in families for j in DESIRED[i,:]])
    # 先满足preference_cost最小
    solver.Minimize(preference_cost)
    status = solver.Solve()
    print('status=',status)
    
    # Solve的结果,x[i,j]决策变量代表第i个family在第j天是否参观
    result = [(i,j,x[i,j].solution_value()) for i in families for j in DESIRED[i,:]
                 if x[i,j].solution_value() > 0]
    
    # 得到参观日期的安排
    df = pd.DataFrame(result,columns=['family_id','day','result'])
    return df

In [28]:
families = unassigned_df['family_id'].unique()
result=solveIP(families,min_occupancy,max_occupancy)
result

status= 0


,family_id,day,result
0,97,95,1.0
1,233,91,1.0
2,244,38,1.0
3,248,32,1.0
4,295,46,1.0
...,...,...,...
58,4613,69,1.0
59,4886,98,1.0
60,4918,58,1.0
61,4936,73,1.0


In [29]:
# 把结果拼接到一起
res=pd.concat([assigned_df[['family_id','day','result']],result])
res['day'] = res['day']+1
res

,family_id,day,result
0,0,52,1.0
1,1,26,1.0
2,2,100,1.0
3,3,2,1.0
4,4,53,1.0
...,...,...,...
58,4613,70,1.0
59,4886,99,1.0
60,4918,59,1.0
61,4936,74,1.0


In [30]:
res = res.sort_values(by=['family_id'],ascending=True)[['family_id','day']]
res.columns=['family_id','assigned_day']
res.to_csv('baseline_santa',index=False)
res

,family_id,assigned_day
0,0,52
1,1,26
2,2,100
3,3,2
4,4,53
...,...,...
5072,4995,16
5073,4996,88
5074,4997,32
5075,4998,92


# 结果评估

In [31]:
from numba import njit
# 预编译装饰器（即时编译器,编译到内存中，作用类似于缓存提高运行效率）
@njit(fastmath=True)
def pcost(prediction):
    penalty = 0
    daily_occupancy = np.zeros(N_DAYS)
    # 统计总的pcost和daily_occupancy
    for (i,p) in enumerate(prediction):
        # 得到家庭人数
        num = FAMILY_SIZE[i]
        # 第i个家庭，第p天访问时的pcost
        penalty += pcost_mat[i,p-1]
        # 计算第p天的人数
        daily_occupancy[p-1] += num 
    # print(penalty)
    return penalty,daily_occupancy

@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost = 0
    # 是否在约束范围外面
    check = False

    for day in range(N_DAYS):
        # 1-100
        if day+1 == 100:
            n1 = int(daily_occupancy[day])
        else:
            # 得到前一天的人数
            n1 = int(daily_occupancy[day+1])
        # 得到当天的人数
        n = int(daily_occupancy[day])
        # 判断是否在MIN_OCCUPANCY和MAX_OCCUPANCY之间
        if (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY) or (n1 > MAX_OCCUPANCY) or (n1 < MIN_OCCUPANCY) :
            check = True
            break
        accounting_cost += acost_mat[n1-1,n-1]
        
    return accounting_cost,check

@njit(fastmath=True)
def cost_function(prediction):
    # 先统计pcost和daily_occupancy
    penalty,daily_occupancy = pcost(prediction)
    accounting_cost,check = acost(daily_occupancy)
    if check:
        return 9999999
    else:
        return penalty + accounting_cost

In [32]:
prediction = res.assigned_day.values
cost = cost_function(prediction)
cost

78454.51836573202

### find_better
* 1)先把完整的cost计算出来 => 完整结果评估：mrtric(pcost+acost)
* 2)对于每个family_id,查看choice可否切换为其它choice => 采用新的结果评估，计算是否小于当前最优解
* 3)不断循环2)，直至无法更新score

In [33]:
# 替换法，更换family_id的choice，查找更好的score
def find_better(pred):
    # 得到原始的score
    score = cost_function(pred)
    # 设置为无穷大
    ori_score = np.inf
    # 如果找不到更新，则退出
    while score < ori_score:
        ori_score = score
        for family_id in range(N_FAMILY):
            # 10个choice
            for pick in range(10):
                # 得到family_id在choice pick的日期
                day = DESIRED[family_id,pick]
                # 记录原来的选择
                oldvalue = pred[family_id]
                # 更换选择
                pred[family_id] = day
                # 重新计算分数
                new_score = cost_function(pred)
                # 如果比原来分数小，那么更新choice成功
                if new_score < score:
                    score = new_score
                # 不比原来小则设置为原来的oldvalue
                else:
                    pred[family_id] = oldvalue
        print('score=',score)
    return pred

In [34]:
prediction = find_better(prediction)

score= 76978.69330012862
score= 76888.84243746771
score= 76888.84243746771


In [35]:
res['assigned_day'] = prediction
res.to_csv('baseline_santa2.csv',index=False)
res

,family_id,assigned_day
0,0,52
1,1,26
2,2,100
3,3,2
4,4,53
...,...,...
5072,4995,16
5073,4996,88
5074,4997,32
5075,4998,92
